<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/Produ%C3%A7%C3%A3o_Commodities_TCC_projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparação de bibliotecas

In [ ]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importando as bibliotecas
from datetime import datetime
import re
import pandas as pd
import pandera as pa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.cloud import storage
import os
import requests
from io import BytesIO
warnings.filterwarnings("ignore")

##Conexões com GCP e Spark

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Delimitando a exposição do df
pd.set_option('display.max_columns',10000)

In [ ]:
# Confiurando a chave de segurança do GCP

serviceAccount = '/content/loyal-optics-377819-ebe367c145ff.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('tcc_soulcode_g3')
bucket.blob('tcc_soulcode_g3')
path = 'gs://tcc_soulcode_g3/brutos_producao/producao_commodities_2010_2022.xlsx'

In [ ]:
# Leitura de df das abas da planilha de origem
df = pd.read_excel(path, sheet_name = 3) # Área plantada (Hectares)
#df1 = pd.read_excel(path, sheet_name = 1) # Área colhida (Hectares)
#df2 = pd.read_excel(path, sheet_name = 2) # Produção (Toneladas)
#df3 = pd.read_excel(path, sheet_name = 3) # Variável - Rendimento médio (Quilogramas por Hectare)
tratado = 'producao_aba_3'

## Análise exploratória

In [ ]:
# Vizualização do df
df

,"Tabela 6588 - Série histórica da estimativa anual da área plantada, área colhida, produção e rendimento médio dos produtos das lavouras",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261,Unnamed: 262,Unnamed: 263,Unnamed: 264,Unnamed: 265,Unnamed: 266,Unnamed: 267,Unnamed: 268,Unnamed: 269,Unnamed: 270,Unnamed: 271,Unnamed: 272,Unnamed: 273,Unnamed: 274,Unnamed: 275,Unnamed: 276,Unnamed: 277,Unnamed: 278,Unnamed: 279,Unnamed: 280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292,Unnamed: 293,Unnamed: 294,Unnamed: 295,Unnamed: 296,Unnamed: 297,Unnamed: 298,Unnamed: 299,Unnamed: 300,Unnamed: 301,Unnamed: 302,Unnamed: 303,Unnamed: 304,Unnamed: 305,Unna

In [ ]:
# Criação de um backup
df_back = df

##Tratamento

In [ ]:
# Transpondo a tabela (tranformando as linhas em colunas e vice versa)
df_tratado = df.transpose().reset_index(drop=True)

In [ ]:
# Vizualização do df transposto
df_tratado

,0,1,2,3,4,5,6
0,Variável - Rendimento médio (Quilogramas por H...,Grande Região,NaN,NaN,Sul,Centro-Oeste,Fonte: IBGE - Levantamento Sistemático da Prod...
1,NaN,Mês x Produto das lavouras,janeiro 2010,Total,..,..,NaN
2,NaN,NaN,NaN,1.8 Feijão (1ª Safra),1472,2106,NaN
3,NaN,NaN,NaN,1.9 Feijão (2ª Safra),1356,1274,NaN
4,NaN,NaN,NaN,1.10 Feijão (3ª Safra),783,2694,NaN
...,...,...,...,...,...,...,...
1088,NaN,NaN,NaN,1.9 Feijão (2ª Safra),1737,1281,NaN
1089,NaN,NaN,NaN,1.10 Feijão (3ª Safra),1333,2715,NaN
1090,NaN,NaN,NaN,1.13 Milho (1ª Safra),5243,7954,NaN
1091,NaN,NaN,NaN,1.14 Milho (2ª Safra),4887,5762,NaN


In [ ]:
# Localização de uma célula específica
valor = df_tratado.at[0,0]

In [ ]:
# Vizualização da célula
valor

'Variável - Rendimento médio (Quilogramas por Hectare)'

In [ ]:
df_tratado = df_tratado.drop(columns=[1,6])

In [ ]:
df_tratado

,0,2,3,4,5
0,Variável - Rendimento médio (Quilogramas por H...,NaN,NaN,Sul,Centro-Oeste
1,NaN,janeiro 2010,Total,..,..
2,NaN,NaN,1.8 Feijão (1ª Safra),1472,2106
3,NaN,NaN,1.9 Feijão (2ª Safra),1356,1274
4,NaN,NaN,1.10 Feijão (3ª Safra),783,2694
...,...,...,...,...,...
1088,NaN,NaN,1.9 Feijão (2ª Safra),1737,1281
1089,NaN,NaN,1.10 Feijão (3ª Safra),1333,2715
1090,NaN,NaN,1.13 Milho (1ª Safra),5243,7954
1091,NaN,NaN,1.14 Milho (2ª Safra),4887,5762


In [ ]:
df_tratado = df_tratado.rename(columns={0: 'definicao',
                                        2: 'periodo',
                                        3: 'commodities',
                                        4: 'regiao_sul',
                                        5: 'regiao_centro_oeste'
                                        })

In [ ]:
# Definindo varável com uma string
definicao = 'definicao'

In [ ]:
# Renomear a coluna
df_tratado[definicao] = valor

In [ ]:
# Vizualização do df_tratado
df_tratado

,definicao,periodo,commodities,regiao_sul,regiao_centro_oeste
0,Variável - Rendimento médio (Quilogramas por H...,NaN,NaN,Sul,Centro-Oeste
1,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,Total,..,..
2,Variável - Rendimento médio (Quilogramas por H...,NaN,1.8 Feijão (1ª Safra),1472,2106
3,Variável - Rendimento médio (Quilogramas por H...,NaN,1.9 Feijão (2ª Safra),1356,1274
4,Variável - Rendimento médio (Quilogramas por H...,NaN,1.10 Feijão (3ª Safra),783,2694
...,...,...,...,...,...
1088,Variável - Rendimento médio (Quilogramas por H...,NaN,1.9 Feijão (2ª Safra),1737,1281
1089,Variável - Rendimento médio (Quilogramas por H...,NaN,1.10 Feijão (3ª Safra),1333,2715
1090,Variável - Rendimento médio (Quilogramas por H...,NaN,1.13 Milho (1ª Safra),5243,7954
1091,Variável - Rendimento médio (Quilogramas por H...,NaN,1.14 Milho (2ª Safra),4887,5762


In [ ]:
# Excluir/dropar a coluna de index
df_tratado = df_tratado.drop(df_tratado.index[0])

In [ ]:
#Backup de df_tratado
df_backup = df_tratado

In [ ]:
#Rodar caso ocorra o erro
df_tratado = df_backup

In [ ]:
# Adicionar uma linha para usar de referência no tratamento e depois excluir
nova_linha = ['bla'] * len(df_tratado.columns)
df_tratado.loc[len(df_tratado) + 1] = nova_linha

In [ ]:
# Vizualização do df_tratado
df_tratado

,definicao,periodo,commodities,regiao_sul,regiao_centro_oeste
1,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,Total,..,..
2,Variável - Rendimento médio (Quilogramas por H...,NaN,1.8 Feijão (1ª Safra),1472,2106
3,Variável - Rendimento médio (Quilogramas por H...,NaN,1.9 Feijão (2ª Safra),1356,1274
4,Variável - Rendimento médio (Quilogramas por H...,NaN,1.10 Feijão (3ª Safra),783,2694
5,Variável - Rendimento médio (Quilogramas por H...,NaN,1.13 Milho (1ª Safra),5521,5682
...,...,...,...,...,...
1089,Variável - Rendimento médio (Quilogramas por H...,NaN,1.10 Feijão (3ª Safra),1333,2715
1090,Variável - Rendimento médio (Quilogramas por H...,NaN,1.13 Milho (1ª Safra),5243,7954
1091,Variável - Rendimento médio (Quilogramas por H...,NaN,1.14 Milho (2ª Safra),4887,5762
1092,Variável - Rendimento médio (Quilogramas por H...,NaN,1.15 Soja,1867,3318


# Lógica de transposição de linhas

In [ ]:
# Lógica para preencher as linhas vazias que surgiram depois da transposição de coluna
for i in range(1, len(df_tratado)):
    if not pd.isnull(df_tratado['periodo'][i]):  # verifica se o valor não é nulo
        substitui = df_tratado['periodo'][i]  # atualiza o valor da variável substitui
        
    elif substitui is not None:  # verifica se a variável substitui foi definida
        df_tratado['periodo'][i] = substitui  # preenche o valor nulo com a variável substitui
        
    else:  # se a variável substitui ainda não foi definida, continue o loop
        continue

    if substitui == 'bla':  # se a variável substitui for 'bla', interrompa o loop
        break

In [ ]:
# Nomeando a variável da linha 'bla' para excluir na sequência
linhas_bla = df_tratado.loc[df_tratado['definicao'] == 'bla']

In [ ]:
# Excluir a linha de referência da lógica
df_tratado = df_tratado.drop(linhas_bla.index)

In [ ]:
# Vizualização do df_tratado
df_tratado

,definicao,periodo,commodities,regiao_sul,regiao_centro_oeste
1,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,Total,..,..
2,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,1.8 Feijão (1ª Safra),1472,2106
3,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,1.9 Feijão (2ª Safra),1356,1274
4,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,1.10 Feijão (3ª Safra),783,2694
5,Variável - Rendimento médio (Quilogramas por H...,janeiro 2010,1.13 Milho (1ª Safra),5521,5682
...,...,...,...,...,...
1088,Variável - Rendimento médio (Quilogramas por H...,dezembro 2022,1.9 Feijão (2ª Safra),1737,1281
1089,Variável - Rendimento médio (Quilogramas por H...,dezembro 2022,1.10 Feijão (3ª Safra),1333,2715
1090,Variável - Rendimento médio (Quilogramas por H...,dezembro 2022,1.13 Milho (1ª Safra),5243,7954
1091,Variável - Rendimento médio (Quilogramas por H...,dezembro 2022,1.14 Milho (2ª Safra),4887,5762


In [ ]:
# Substituindo os nomes dos meses dentro de 'periodo' 
# por números para viabilizar a conversão do tipo de coluna

df_tratado = df_tratado.replace({'janeiro ': '01/01/', 
                                 'fevereiro ': '01/02/',
                                 'março ': '01/03/',
                                 'abril ': '01/04/',
                                 'maio ': '01/05/',
                                 'junho ': '01/06/',
                                 'julho ': '01/07/',
                                 'agosto ': '01/08/',
                                 'setembro ': '01/09/',
                                 'outubro ': '01/10/',
                                 'novembro ': '01/11/',
                                 'dezembro ': '01/12/',
                                 }, regex=True)

In [ ]:
# Vizualização do df_tratado
df_tratado

,definicao,periodo,commodities,regiao_sul,regiao_centro_oeste
1,Variável - Rendimento médio (Quilogramas por H...,01/01/2010,Total,..,..
2,Variável - Rendimento médio (Quilogramas por H...,01/01/2010,1.8 Feijão (1ª Safra),1472,2106
3,Variável - Rendimento médio (Quilogramas por H...,01/01/2010,1.9 Feijão (2ª Safra),1356,1274
4,Variável - Rendimento médio (Quilogramas por H...,01/01/2010,1.10 Feijão (3ª Safra),783,2694
5,Variável - Rendimento médio (Quilogramas por H...,01/01/2010,1.13 Milho (1ª Safra),5521,5682
...,...,...,...,...,...
1088,Variável - Rendimento médio (Quilogramas por H...,01/12/2022,1.9 Feijão (2ª Safra),1737,1281
1089,Variável - Rendimento médio (Quilogramas por H...,01/12/2022,1.10 Feijão (3ª Safra),1333,2715
1090,Variável - Rendimento médio (Quilogramas por H...,01/12/2022,1.13 Milho (1ª Safra),5243,7954
1091,Variável - Rendimento médio (Quilogramas por H...,01/12/2022,1.14 Milho (2ª Safra),4887,5762


In [ ]:
# Limpando os textos desnecessários da coluna 'commodities' para eventual junção (groupby)
df_tratado = df_tratado.replace({'1.8 ':'',
                                 '1.9 ':'',
                                 '1.10 ':'',
                                 '1.13 ':'',
                                 '1.14 ':'',
                                 '1.15 ':'',
                                 ' \(1ª Safra\)': '',
                                 ' \(2ª Safra\)': '',
                                 ' \(3ª Safra\)': ''},regex=True)

In [ ]:
# Vizualização do df_tratado
df_tratado

In [ ]:
#Converter coluna 'periodo' para date
df_tratado['periodo'] = pd.to_datetime(df_tratado['periodo'], format='%d/%m/%Y')

In [ ]:
#conversão do tipo das colunas
df_tratado['regiao_sul'] = pd.to_numeric(df_tratado['regiao_sul'], errors='coerce').astype('Int64')
df_tratado['regiao_centro_oeste'] = pd.to_numeric(df_tratado['regiao_centro_oeste'], errors='coerce').astype('Int64')

In [ ]:
df_tratado.dtypes

definicao                      object
periodo                datetime64[ns]
commodities                    object
regiao_sul                      Int64
regiao_centro_oeste             Int64
dtype: object

## Validação de dados

In [ ]:
#Criação do schema pandera

schema = pa.DataFrameSchema(
    columns = {
        'definicao':pa.Column(pa.String),
        'periodo	':pa.Column(pa.DateTime),
        'commodities':pa.Column(pa.String),
        'regiao_sul':pa.Column(pa.Int),
        'regiao_centro_oeste':pa.Column(pa.Int)
    }
)

##Load (Subindo df tratado para GCP)

In [ ]:
# SALVANDO OS DADOS NO BUCKET
gcs_path = f'gs://tcc_soulcode_g3/tratados_producao/{tratado}_Tratado.csv'
df_tratado.to_csv(gcs_path, sep = ';', index = False)